# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data_file = pd.read_csv("city_data.csv")
data_file.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Bembèrèkè,78,BJ,1592771396,78,10.23,2.66,77.97,9.10
1,1,Clyde River,1,CA,1592771127,81,70.47,-68.59,44.60,5.82
2,2,Kruisfontein,0,ZA,1592771129,88,-34.00,24.73,53.78,5.77
3,3,Richards Bay,0,ZA,1592771396,64,-28.78,32.04,62.29,6.46
4,4,Saint-Pierre,20,RE,1592771396,82,-21.34,55.48,66.20,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
location = data_file[["Lat","Lng"]]
humidity = data_file["Humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
markers = gmaps.heatmap_layer(location)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# A max temperature lower than 80 degrees but higher than 70.
filtered_data = data_file.loc[(data_file["Max Temp"]>70) & (data_file["Max Temp"]<80)]

# Wind speed less than 10 mph
filtered_data = filtered_data.loc[(filtered_data["Wind Speed"]<10)]

# Zero cloudiness
filtered_data = filtered_data.loc[(filtered_data["Cloudiness"]==0)]

# Drop any null values and display the data
filtered_data.dropna()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
21,21,Nador,0,MA,1592771150,69,35.17,-2.93,77.00,3.36
94,94,Caravelas,0,BR,1592771131,68,-17.71,-39.25,72.64,4.00
160,160,Abu Samrah,0,SY,1592771411,40,35.30,37.18,73.63,5.84
213,213,Chaman,0,PK,1592771415,14,30.92,66.45,76.86,4.29
218,218,Deputatsky,0,RU,1592771145,28,69.30,139.90,77.00,1.54
267,267,Herat Province,0,AF,1592771421,16,34.50,62.00,78.39,4.70
271,271,Ouango,0,CF,1592771421,95,4.31,22.56,70.84,2.48
345,345,Popondetta,0,PG,1592771159,93,-8.75,148.25,72.09,2.64
347,347,Fukuechō,0,JP,1592771428,84,32.69,128.84,72.52,7.99
350,350,Ýpsonas,0,CY,1592771429,83,34.69,32.96,73.40,1.12


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = filtered_data.copy()

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
21,21,Nador,0,MA,1592771150,69,35.17,-2.93,77.00,3.36,
94,94,Caravelas,0,BR,1592771131,68,-17.71,-39.25,72.64,4.00,
160,160,Abu Samrah,0,SY,1592771411,40,35.30,37.18,73.63,5.84,
213,213,Chaman,0,PK,1592771415,14,30.92,66.45,76.86,4.29,
218,218,Deputatsky,0,RU,1592771145,28,69.30,139.90,77.00,1.54,


In [9]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

In [ ]:
for index,row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # add location to params
    params["location"] = f"{lat},{lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, params = params).json()
    #print(json.dumps(response,indent = 4, sort_keys =True))
    
    try:
        hotel_df[index, "Hotel Name"] = response["results"][0]["name"] 
    except:
        print("Missing field, skipping...")

hotel_df

Missing field, skipping...


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [50]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))